Libraries

In [1]:
# Standard libraries
from cellpose import models, core, plot, utils
import nibabel as nib

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Custom packages
from auxiliary.datasets import HtDataset
from auxiliary.imaging import read_nii

# Configurations
use_gpu = core.use_gpu()
print(f"GPU activated: {use_gpu}")

from cellpose.io import logger_setup
logger_setup();

GPU activated: True
2024-07-19 15:48:32,888 [INFO] WRITING LOG OUTPUT TO /home/imarcoss/.cellpose/run.log
2024-07-19 15:48:32,889 [INFO] 
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.10.14 
torch version:  	2.3.1+cu118


In [2]:
ds = HtDataset()
ds.check_specimens()

img_idx = 0
img_path = ds.missing_nuclei[img_idx]
img_path_out = ds.missing_nuclei_out[img_idx]

img = read_nii(img_path)
print(img.shape)

AttributeError: 'HtDataset' object has no attribute 'specimens'

In [ ]:
model = models.Cellpose(gpu=use_gpu, model_type='nuclei')

In [10]:
img_equalized = cv2.equalizeHist(img)

In [ ]:
for i in [100, 270, 400]:
    cv2.imshow('image', img[:, :, i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
for i in [100, 270, 400]:
    cv2.imshow('image', img_equalized[:, :, i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
masks, flows, styles, diams = model.eval(
    img_equalized, diameter=25, channels=[0, 0], 
    do_3D=True, normalize=True
)

# masks_stitched, flows_stitched, styles_stitched, _ = model.eval(
# img_equalized, channels=[0, 0], diameter=25, do_3D=False, 
# stitch_threshold=0.5, normalize = True
# )

In [ ]:
plt.figure(figsize=(12, 6))

for i, iplane in [100, 270, 400]:
  img0 = plot.image_to_rgb(img_equalized[iplane, [1, 0]].copy(), channels=[0, 0])
  
  plt.subplot(1,8,i+1)
  
  outlines = utils.masks_to_outlines(masks[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)